In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import statsmodels.api as sm
import pickle as pkl
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

<div dir="rtl">

# פרויקט מסכם תשפ"ו סמסמטר א
    
### בפרויקט זה ננתח חברת כימיקלים. בחברה יש שני סוגים של כימיקלים  : Sulfuric_Acid ו - Hydrochloric_acid. 
    
### כל עובד מתמחה אך ורק בסוג אחד של כימיקל. לכן הוא לא יעבוד עם ההסוג השני בשום צורה ואינו מוסמך לכך.


### בפרויקט נרצה להתמקם בתאונות עבודה העלולות להתרחש במפעל.


### החברה ממוקמת בישראל ויש לה מפעלים המפוזרים ברחבי המדינה.
    
### בנוסף, כל עובד עובר מבחני בטיחות ומקבל עליהם ציון - מספר בין 1 ל 10. כאשר 10 הכי גבוה. 
        
## Dataset חלק א':
    
###  במפעל ישנם חומרים כימיים: 'Hydrochloric_acid' ו 'Sulfuric Acid'
    
###  לאחר  השימוש באחד החומרים נשאר שאריות של החומר וקשה להבחין איזה חומר זה בין השניים. 
    
### מה שכן אנחנו יודעים זה את הטמפרטורה של החומר והאם התגבשו גושים בחומר או לא. שתי תכונות אלו יכולות להעיד על סוג החומר. 
    
### בdataset יש את העמודות הבאות: 
    

### Intercept: A column of ones 
### temp: the chimical temperture 
### rigid_bool: Categorial  - true if there are some lumps. 
### Hydrochloric_acid: True if Hydrochloric acid, False if Sulfuric Acid

In [25]:
num_group = 1
df_part_1 = pkl.load( open('../data_logit_poisson/project_part_1_group_'+str(num_group)+'.pkl', 'rb'))
df_part_1.head()

,Intercept,temp,rigid_bool,Hydrochloric_acid
0,1.0,19.404139,True,False
1,1.0,24.819121,True,False
2,1.0,34.663911,False,False
3,1.0,22.853774,False,True
4,1.0,39.344357,True,True


<div dir="rtl">

## חלק א:
    

    
### 1. נא לבנות מודל רגרסיה לוגיסטית שחוזה את ההסתברות שהחומר הוא mathane.  יש להציג את המשוואה הסופית לתחזית.
    
### כעת נרצה לסווג האם חומר הוא  'Hydrochloric_acid' או 'Sulfuric Acid'
### לשם אחידות - נגדיר Hydrochloric_acid  הוא 1 ואילו Sulfuric Acid הוא 0
    
### 2. האם קיים ערך סף לסיווג כך שהsensativity וגם הspecaficity מעל 0.6? אם כן, מהו תחום ערכי הסף המקיימים זאת?
    
### 3.   מהו הthreshold שממקסם את הaccuracy? אם יש יותר מאחד נא לפרט את התחום
    
### מה ההסתברות שהחומר הוא Hydrochloric_acid בשלושת המקרים הבאים: 
    
### א. טמפרטורה 23.2 ונשאר שאריות
    
### ב. טמפרטורה 24.2 ונשאר שאריות
    
### ג. טמפרטורה 25.2 ונשאר שאריות
    
### 4.  מהו הlog odd ratio בין ב' לא' ובין ג' לב' ובין ג' לא'? 
    
### האם שלוש הערכים שקיבלתם שווים? האם חלקם שווים? נא הסבר.י את משמעות התוצאה שקיבלתם.

In [26]:
num_group = 1

df_part_2 = pkl.load( open('../data_logit_poisson/project_part_2_group_'+str(num_group)+'.pkl', 'rb'))
df_part_2.head()

,Intercept,age,safety_guidence_score,factory,chemical,time_in_factory,num_accidents
0,1.0,42,1,Makeret_Batya,Sulfuric_Acid,8,4
1,1.0,30,4,Ariel,Sulfuric_Acid,6,1
2,1.0,41,7,Haifa,Sulfuric_Acid,7,6
3,1.0,33,7,Haifa,Sulfuric_Acid,7,0
4,1.0,23,10,Jerusalem,Sulfuric_Acid,6,10


<div dir="rtl">

## חלק ב:
    
### dataset עבור חלק ב'
        
### כל שורה בעצם נותנת לנו נתונים לכמה זמן עובד נמצא בחברה, איזה סוג כימיקל הוא עובד איתו, את גיל העובד מיקום המפעל  ציוני במחבני בטיחות וכמות התאונות שהיו לו במשך שנותיו במפעל.

### Intercept:  a column of ones 
### age: the age of the workers in years
### safety_guidence_score: the score of the safety test a worker is taking - can be any number between 0 to 10
### factory: categorial variabal which indicate the factory
### chemical: there are two chemicals in which workers are working with: Hydrochloric_acid and Sulfuric_Acid
### time_in_factory: experience in years
## num_accident: total cumulative number of accidents a worker had during their time in the factory


In [28]:
df_part_2 = pkl.load( open('../data_logit_poisson/project_part_2_group_'+str(num_group)+'.pkl', 'rb'))
df_part_2.head()

,Intercept,age,safety_guidence_score,factory,chemical,time_in_factory,num_accidents
0,1.0,42,1,Makeret_Batya,Sulfuric_Acid,8,4
1,1.0,30,4,Ariel,Sulfuric_Acid,6,1
2,1.0,41,7,Haifa,Sulfuric_Acid,7,6
3,1.0,33,7,Haifa,Sulfuric_Acid,7,0
4,1.0,23,10,Jerusalem,Sulfuric_Acid,6,10


<div dir="rtl">

    
###  מטלות:
     
### 1. נא להציג bar chart של המשתנה הקטגוריאלי factory כאשר נרצה להציג מספר תאונות בממוצע לשנת עבודה עבור כל מפעל. 
    
### האם לפי התוצאות שקיבלתם נראה כי משתנה זה יהיה מובהק וייכנס למודל?    נמק.י

        
### 2. יש כעת לעשות רגרסיה פואסונית כולל למודל ולבסוף להציג את המשוואה הסופית
### הערה: במשתנים הקטגוריאלים יש להזין את טורונטו Hydrochloric_acid כמשתני בסיס למודל. 
    
### 3. האם רגרסיה פואוסנית מתאימה כאן? נמק.י יש לבדוק - 4 דרכים שונות 

### 4. באחת הבדיקות להנחות המודל נרצה להראות שכ95 אחוז מהשגיאות הם +- 2 סטיות תקן. למה בדיקה זו עוזרת להבין את הנחות המודל? מה בדיוק אנחנו מניחים שאנחנו עושים רגרסיה פואסונית?
    
### 5. בבדיקה של הנחת המודל שהתוחלת שווה לשונות אנחנו מחלקים את הנתונים לbuckets ובודקים בכל אחד שזה מתקיים, מדוע? למה לא פשוט לחשב את השונות והתוחלת של המשתנה המוסבר בdataset ולראות האם שווים?
    
### 6. אם בסעיף 4 התשובה הייתה לא, נא לעשות רגרסיה בינומית שלילית ולבדוק אם היא מתאימה כאן.
###  אם בסעיף 4 התשובה הייתה כן נא לחשב את הIRR של אחד המשתנים הרציפים ולהסביר מה המשמעות שלו במונחים של מספר תאונות

### 7. לפי משוואת רגרסיה שקיבלתם, מהו מספר התאונות הצפוי לעובד בן 30 שקיבלת 8 במבחן בטיחות עבור 3 שנים עבודה במפעל. לגבי משתנים קטגוריאלים תשתמשו רק בקטגוריות בסיס. 
    
### 8. מה המשמעות של התוצאה שקיבלתם? מה המספר הזה מייצג בדיוק?  
     
### 9. predictions:    נא  להציג גרף תלת מימדי של מספר תאונות עבור פרק זמן של 8 שנים, במפעל במזכרת בתיה עבור החומר Hydrochloric_acid, כפונקציה של הגיל ותוצאת בחינה בטיחות. 

### 10. שאלה תיאורטית: מה ההבדל בין רגרסיה פואסונית לרגרסיה ליניארית מבחינת מציאת ערכי הפרמטרים? במה רגרסיה ליניארית עדיפה בהיבט  הזה? 
    


# Good luck!!!